In [1]:
import pandas as pd
import pyspark

In [2]:
#Cargamos el archivo a utilizar 
events = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

In [105]:
#Cargamos el archivo a utilizar 
labels = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

#Hay mucha cantidad de valores de 0 (no conversiones)

In [108]:
# armo df con registros completos clasificados
train = events.merge(labels, on='person', how='right')

In [110]:
# armo df con registros a predecir unicamente
to_predict = events[~events.person.isin(labels.person)]

In [113]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Data columns (total 2 columns):
person    19414 non-null object
label     19414 non-null int64
dtypes: int64(1), object(1)
memory usage: 303.4+ KB


In [125]:
personas_con_labels = events.join(labels.set_index('person'), on='person', how='right')

In [126]:
personas_con_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1171886 entries, 1 to 2341548
Data columns (total 36 columns):
timestamp                   1171886 non-null datetime64[ns]
event                       1171886 non-null object
person                      1171886 non-null object
url                         94875 non-null object
sku                         665336 non-null float64
model                       665767 non-null object
condition                   665336 non-null object
storage                     665336 non-null object
color                       665336 non-null object
skus                        249587 non-null object
search_term                 55774 non-null object
staticpage                  5660 non-null object
campaign_source             94940 non-null object
search_engine               52829 non-null object
channel                     102299 non-null object
new_vs_returning            102299 non-null object
city                        102299 non-null object
region        

In [120]:
personas_con_labels.groupby('person')

Quiero predecir una quincena.

In [123]:
events['timestamp'] = pd.to_datetime(events['timestamp'])
events['mes'] = events['timestamp'].dt.month
events['dia'] = events['timestamp'].dt.day

In [124]:
events['q1'] = ((events['mes']==1) & (events['dia']>1) & (events['dia']<16))
events['q2'] = ((events['mes']==1) & (events['dia']>15) & (events['dia']<32))
events['q3'] = ((events['mes']==2) & (events['dia']>1) & (events['dia']<16))
events['q4'] = ((events['mes']==2) & (events['dia']>15) & (events['dia']<29))
events['q5'] = ((events['mes']==3) & (events['dia']>1) & (events['dia']<16))
events['q6'] = ((events['mes']==3) & (events['dia']>15) & (events['dia']<32))
events['q7'] = ((events['mes']==4) & (events['dia']>1) & (events['dia']<16))
events['q8'] = ((events['mes']==4) & (events['dia']>15) & (events['dia']<31))
events['q9'] = ((events['mes']==5) & (events['dia']>1) & (events['dia']<16))
events['q10'] = ((events['mes']==5) & (events['dia']>15) & (events['dia']<32))

In [127]:
ap = pd.DataFrame()
ap['person'] = events['person']
ap['event'] = events['event']

In [151]:
# Llevamos los datos a una matriz de personas y eventos, formato requerido por la libreria para aplicar el algoritmo apriori:
ap = pd.crosstab(ap['person'], ap['event'])

In [152]:
# Tenemos los datos presentados de la forma que requiere la libreria para aplicar el algoritmo apriori
ap.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,0,0,3,0,1,0,0,0,0,0,2
00091926,15,25,2,0,0,0,0,0,0,372,34
00091a7a,1,5,0,0,0,0,0,0,0,3,1
000ba417,1,24,6,1,14,0,1,0,0,153,6
000c79fe,1,0,1,0,1,0,1,9,0,3,1


In [153]:
# Reducimos uso de memoria (es una matriz de categorias x postulantes, donde cada celda indica si existio o no una postulacion a esa categoria, con 1 o 0)
ap = ap.astype('bool')
ap.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 0008ed71 to fffd1246
Data columns (total 11 columns):
ad campaign hit      38829 non-null bool
brand listing        38829 non-null bool
checkout             38829 non-null bool
conversion           38829 non-null bool
generic listing      38829 non-null bool
lead                 38829 non-null bool
search engine hit    38829 non-null bool
searched products    38829 non-null bool
staticpage           38829 non-null bool
viewed product       38829 non-null bool
visited site         38829 non-null bool
dtypes: bool(11)
memory usage: 720.5+ KB


In [154]:
# Algoritmo apriori para detectar las n-uplas de categorias mas frecuentes a las que un mismo postulante se postula. En este caso para el analisis
# usamos max_len=2 asi limitamos al algoritmo a tomar como maximo 2-uplas (duplas). Usamos un soporte minimo (frecuencia de aparicion sobre el total 
# de registros) min_support=0.0 para que muestre todas las duplas posibles:
from mlxtend.frequent_patterns import apriori 
apriori_result = apriori(ap, min_support=0.0, use_colnames=True, max_len=2)

In [155]:
apriori_result.head()

,support,itemsets
0,0.821551,(ad campaign hit)
1,0.495223,(brand listing)
2,0.845579,(checkout)
3,0.110562,(conversion)
4,0.648613,(generic listing)


In [157]:
apriori(ap, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.821551,(ad campaign hit)
1,0.845579,(checkout)
2,0.648613,(generic listing)
3,0.671689,(search engine hit)
4,0.956244,(viewed product)
5,0.984882,(visited site)
6,0.699374,"(checkout, ad campaign hit)"
7,0.807232,"(ad campaign hit, viewed product)"
8,0.819104,"(ad campaign hit, visited site)"
9,0.816967,"(checkout, viewed product)"


In [171]:
frequent_itemsets = apriori(ap, min_support=0.0, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.821551,(ad campaign hit),1
1,0.495223,(brand listing),1
2,0.845579,(checkout),1
3,0.110562,(conversion),1
4,0.648613,(generic listing),1
5,0.014912,(lead),1
6,0.671689,(search engine hit),1
7,0.337196,(searched products),1
8,0.104407,(staticpage),1
9,0.956244,(viewed product),1


In [173]:
frequent_itemsets['conversion'] = frequent_itemsets['itemsets'].apply(lambda x: 'conversion' in x)

In [174]:
frequent_itemsets[ (frequent_itemsets['conversion']) ]

,support,itemsets,length,conversion
3,0.110562,(conversion),1,True
13,0.095084,"(conversion, ad campaign hit)",2,True
22,0.080378,"(conversion, brand listing)",2,True
30,0.109557,"(conversion, checkout)",2,True
38,0.099513,"(conversion, generic listing)",2,True
39,0.005666,"(conversion, lead)",2,True
40,0.088516,"(conversion, search engine hit)",2,True
41,0.061732,"(conversion, searched products)",2,True
42,0.040949,"(staticpage, conversion)",2,True
43,0.108759,"(conversion, viewed product)",2,True


In [175]:
frequent_itemsets.conversion.value_counts()

True     1024
False    1023
Name: conversion, dtype: int64

In [188]:
ap_l = pd.crosstab(personas_con_labels['person'], personas_con_labels['event'])
ap_l.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,0,0,3,0,1,0,0,0,0,0,2
000c79fe,1,0,1,0,1,0,1,9,0,3,1
001802e4,5,0,1,0,4,0,0,4,0,4,1
0019e639,29,165,15,2,28,0,13,11,0,189,19
001b0bf9,0,1,2,1,1,0,0,0,0,2,0


In [183]:
ap_l = ap_l.astype('bool')

In [184]:
from mlxtend.frequent_patterns import apriori 
apriori_result = apriori(ap_l, min_support=0.0, use_colnames=True, max_len=2)

In [186]:
apriori_result['conversion'] = frequent_itemsets['itemsets'].apply(lambda x: 'conversion' in x)

In [192]:
apriori_result[ apriori_result['conversion'] ]

,support,itemsets,conversion
3,0.111466,(conversion),True
13,0.094880,"(conversion, ad campaign hit)",True
22,0.081230,"(conversion, brand listing)",True
30,0.110487,"(conversion, checkout)",True
38,0.100340,"(conversion, generic listing)",True
39,0.005615,"(conversion, lead)",True
40,0.088184,"(conversion, search engine hit)",True
41,0.062326,"(conversion, searched products)",True
42,0.040898,"(staticpage, conversion)",True
43,0.109921,"(conversion, viewed product)",True


In [4]:
#No funciona
#events_rdd = sc.parallelize(events)  
#events_rdd.take()

In [5]:
#csv = spark.read.csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", header=True, mode="DROPMALFORMED")
events_rdd = spark\
         .read\
         .format("com.databricks.spark.csv")\
         .option("header", "true")\
         .load("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv").rdd   

In [4]:
events_rdd = sc.textFile("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv",8) 

In [9]:
events_rdd

/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [17]:
events_mapeado = events_rdd.map(lambda x: (x[2], x[1]) )

In [18]:
from pyspark.mllib.fpm import FPGrowth

#transactions = events_mapeado.map(lambda line: line.strip().split(' '))
model = FPGrowth.train(events_mapeado, minSupport=0.2, numPartitions=10)
result = model.freqItemsets().collect()
for fi in result:
    print(fi)

Py4JJavaError: An error occurred while calling o40.trainFPGrowthModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1, localhost, executor driver): java.lang.ClassCastException: [Ljava.lang.Object; cannot be cast to java.lang.Iterable
	at org.apache.spark.mllib.api.python.PythonMLLibAPI$$anonfun$9.apply(PythonMLLibAPI.scala:579)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.util.Utils$.getIteratorSize(Utils.scala:1835)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1162)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1162)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:2067)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:2067)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:87)
	at org.apache.spark.scheduler.Task.run(Task.scala:1

In [40]:
def a(x):
    list = []
    list.append(x)

In [101]:
grouped = events.groupby(events['person'], as_index=True)['event']
grouped

In [98]:
diccionario = {}
for group in grouped:
    diccionario[group[0]] = group[1] 

In [99]:
diccionario

{'0008ed71': 1505383           checkout
 1507286           checkout
 1507716           checkout
 2122051    generic listing
 2336760       visited site
 2336761       visited site
 Name: event, dtype: object, '00091926': 129        viewed product
 142        viewed product
 144        viewed product
 145        viewed product
 219        viewed product
 223        viewed product
 225        viewed product
 227        viewed product
 228        viewed product
 229        viewed product
 240        viewed product
 277        viewed product
 278        viewed product
 281        viewed product
 302        viewed product
 303        viewed product
 461        viewed product
 462        viewed product
 463        viewed product
 465        viewed product
 467        viewed product
 21354      viewed product
 21355      viewed product
 21364      viewed product
 21372      viewed product
 21438      viewed product
 21445      viewed product
 21455      viewed product
 21458      viewed produ

In [100]:
grouped.get_group('4886f805')

0             viewed product
132768        viewed product
199884     search engine hit
199951              checkout
199987        viewed product
199990     searched products
200125        viewed product
1765551      generic listing
2147123         visited site
Name: event, dtype: object

In [102]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(diccionario).transform(diccionario)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f
0,True,True,False,False,False,False,False,True,True,False,False,False,False,True,True,False
1,True,True,True,False,False,False,True,False,False,True,False,False,False,False,False,False
2,True,True,False,False,False,False,False,True,False,True,True,False,False,False,False,False
3,True,True,False,False,True,False,False,True,False,False,True,True,False,False,False,False
4,True,False,False,False,False,False,False,True,False,True,False,False,True,False,True,True
5,True,False,False,False,True,False,False,False,False,True,False,False,False,True,True,False
6,True,True,False,False,False,False,True,False,False,True,False,False,False,True,True,False
7,True,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False
8,True,True,False,False,False,False,False,False,True,True,True,False,False,False,True,False
9,True,True,False,False,True,True,True,False,False,False,False,True,True,False,False,False


In [87]:
for group in grouped:
    print( group )
    for x in group:
        print(x[0])

('0008ed71', 1505383           checkout
1507286           checkout
1507716           checkout
2122051    generic listing
2336760       visited site
2336761       visited site
Name: event, dtype: object)
0


KeyError: 0